# Gestion Energétique en Zone Urbaine - Exercice

In [1]:
import os
import pandas as pd

data_folder = "donnees"

os.listdir(data_folder)

['layer_buildings.shx',
 'layer_buildings.sbx',
 'layer_buildings.sbn',
 'consommations_specifiques.csv',
 'layer_buildings.shp',
 'layer_buildings.shp.xml',
 'donnees_batiments.csv',
 'layer_buildings.dbf',
 'layer_buildings.prj',
 'agent_ener_perf.csv']

## Récupération des données
Les données concernant les bâtiments de la zone d'étude se trouvent dans le fichier `donnees_batiments.csv`.

In [2]:
raw_data = pd.read_csv(os.path.join(data_folder, 'donnees_batiments.csv'), index_col=0)
raw_data.head()

,TypeAffectation,PeriodeConstruction,AgentEnergetique,NbrNiveauxChauffes,EmpreinteSol_[m2]
ID,,,,,
1,Immeuble de bureaux,1981-1985,Mazout,3,518
2,Batiment commercial,2011-2015,Mazout,1,1203
3,Immeuble de bureaux,1971-1980,Electricite,3,428
4,Immeuble de bureaux,1981-1985,Gaz naturel,4,985
5,Maison mitoyenne,1981-1985,Gaz naturel,4,1275


Les consommations spécifiques des bâtiments (en kWh/m2/an) sont des valeurs statistiques moyennes standardisées.
Elles seront utilisées pour estimer l'énergie utile nécessaire pour chauffer les bâtiments de la zone d'étude.
Ces données sont présentes dans le fichier `consommations_specifiques.csv`.

In [3]:
c_spec = pd.read_csv(os.path.join(data_folder, 'consommations_specifiques.csv'), index_col=0)
c_spec.head()

,1900-1920,1921-1945,1946-1960,1961-1970,1971-1980,1981-1985,1986-1990,1991-1995,1996-2000,2001-2005,2006-2010,2011-2015
TypeAffectation,,,,,,,,,,,,
Batiment commercial,124,140,140,137,124,101,85,68,68,52,52,39
Batiment culturel,158,179,179,175,158,129,108,87,87,67,67,50
Batiment industriel,137,155,155,151,137,112,94,76,76,58,58,43
Entrepot,107,121,121,118,107,87,73,59,59,45,45,34
Hopital,216,244,244,238,216,176,148,119,119,91,91,68


Les facteurs permettant de calculer l'énergie finale, l'énergie primaire et les rejets de CO2 associés aux besoins de chauffage des bâtiments sont disponibles dans le fichier `agent_ener_perf.csv`.

In [4]:
perf_ae = pd.read_csv(os.path.join(data_folder, 'agent_ener_perf.csv'), index_col=0)
perf_ae.head()

,Factor_E_prim_[ep/ef],CO2_[kgCO2/MJ_ef],Eff_[eu/ef]
AE,,,
Pas d'agent,0.00,0.0000,0.0
Mazout,1.24,0.2952,0.9
Charbon,1.66,0.4320,0.8
Gaz naturel,1.15,0.2412,0.9
Electricite,2.34,0.0800,1.0


## 1. Calcul de la Surface de Référence Energétique (SRE)
Les différents calculs se font sur une copie des données.

$SRE = 0.9 * Empreinte \: au \: sol* Nombre \: de \: niveaux$

In [5]:
data = raw_data.copy()

ratio_sre = 0.9
data["SRE_[m2]"] = ratio_sre * data["EmpreinteSol_[m2]"] * data["NbrNiveauxChauffes"]
data.head()

,TypeAffectation,PeriodeConstruction,AgentEnergetique,NbrNiveauxChauffes,EmpreinteSol_[m2],SRE_[m2]
ID,,,,,,
1,Immeuble de bureaux,1981-1985,Mazout,3,518,1398.6
2,Batiment commercial,2011-2015,Mazout,1,1203,1082.7
3,Immeuble de bureaux,1971-1980,Electricite,3,428,1155.6
4,Immeuble de bureaux,1981-1985,Gaz naturel,4,985,3546.0
5,Maison mitoyenne,1981-1985,Gaz naturel,4,1275,4590.0


## 2. Estimation de l'énergie utile
$E_{utile} = SRE * conso_{specifique}$

In [6]:
def e_utile(row):
    return row["SRE_[m2]"] * c_spec.at[row["TypeAffectation"], row["PeriodeConstruction"]]

data["E_utile_[kWh/an]"] = data.apply(e_utile, axis=1)
data.head()

,TypeAffectation,PeriodeConstruction,AgentEnergetique,NbrNiveauxChauffes,EmpreinteSol_[m2],SRE_[m2],E_utile_[kWh/an]
ID,,,,,,,
1,Immeuble de bureaux,1981-1985,Mazout,3,518,1398.6,176223.6
2,Batiment commercial,2011-2015,Mazout,1,1203,1082.7,42225.3
3,Immeuble de bureaux,1971-1980,Electricite,3,428,1155.6,179118.0
4,Immeuble de bureaux,1981-1985,Gaz naturel,4,985,3546.0,446796.0
5,Maison mitoyenne,1981-1985,Gaz naturel,4,1275,4590.0,711450.0


## 3. Estimation de l'énergie finale
$E_{finale} = \frac{E_{finale}}{rendement_{agent \: énergétique}}$

In [7]:
def e_finale(row):
    return round(row["E_utile_[kWh/an]"] / perf_ae.at[row["AgentEnergetique"], "Eff_[eu/ef]"], 1)

data["E_finale_[kWh/an]"] = data.apply(e_finale, axis=1)
data.head()

,TypeAffectation,PeriodeConstruction,AgentEnergetique,NbrNiveauxChauffes,EmpreinteSol_[m2],SRE_[m2],E_utile_[kWh/an],E_finale_[kWh/an]
ID,,,,,,,,
1,Immeuble de bureaux,1981-1985,Mazout,3,518,1398.6,176223.6,195804.0
2,Batiment commercial,2011-2015,Mazout,1,1203,1082.7,42225.3,46917.0
3,Immeuble de bureaux,1971-1980,Electricite,3,428,1155.6,179118.0,179118.0
4,Immeuble de bureaux,1981-1985,Gaz naturel,4,985,3546.0,446796.0,496440.0
5,Maison mitoyenne,1981-1985,Gaz naturel,4,1275,4590.0,711450.0,790500.0


## 4. Estimation de l'énergie primaire
$E_{primaire} = E_{finale} * facteur_{E_p/E_f}$

In [8]:
def e_primaire(row):
    return round(row["E_finale_[kWh/an]"] * perf_ae.at[row["AgentEnergetique"], "Factor_E_prim_[ep/ef]"], 1)

data["E_primaire_[kWh/an]"] = data.apply(e_primaire, axis=1)
data.head()

,TypeAffectation,PeriodeConstruction,AgentEnergetique,NbrNiveauxChauffes,EmpreinteSol_[m2],SRE_[m2],E_utile_[kWh/an],E_finale_[kWh/an],E_primaire_[kWh/an]
ID,,,,,,,,,
1,Immeuble de bureaux,1981-1985,Mazout,3,518,1398.6,176223.6,195804.0,242797.0
2,Batiment commercial,2011-2015,Mazout,1,1203,1082.7,42225.3,46917.0,58177.1
3,Immeuble de bureaux,1971-1980,Electricite,3,428,1155.6,179118.0,179118.0,419136.1
4,Immeuble de bureaux,1981-1985,Gaz naturel,4,985,3546.0,446796.0,496440.0,570906.0
5,Maison mitoyenne,1981-1985,Gaz naturel,4,1275,4590.0,711450.0,790500.0,909075.0


## 5. Estimation des rejets de CO2
$Rejets_{CO_2} = E_{finale} * facteur_{CO_2}$

In [9]:
def rej_co2(row):
    return round(row["E_finale_[kWh/an]"] * 3.6 * perf_ae.at[row["AgentEnergetique"], "CO2_[kgCO2/MJ_ef]"] * 1e-3, 1)

data["RejCO2_[Tco2/an]"] = data.apply(rej_co2, axis=1)
data.head()

,TypeAffectation,PeriodeConstruction,AgentEnergetique,NbrNiveauxChauffes,EmpreinteSol_[m2],SRE_[m2],E_utile_[kWh/an],E_finale_[kWh/an],E_primaire_[kWh/an],RejCO2_[Tco2/an]
ID,,,,,,,,,,
1,Immeuble de bureaux,1981-1985,Mazout,3,518,1398.6,176223.6,195804.0,242797.0,208.1
2,Batiment commercial,2011-2015,Mazout,1,1203,1082.7,42225.3,46917.0,58177.1,49.9
3,Immeuble de bureaux,1971-1980,Electricite,3,428,1155.6,179118.0,179118.0,419136.1,51.6
4,Immeuble de bureaux,1981-1985,Gaz naturel,4,985,3546.0,446796.0,496440.0,570906.0,431.1
5,Maison mitoyenne,1981-1985,Gaz naturel,4,1275,4590.0,711450.0,790500.0,909075.0,686.4


## 6. Estimation de la puissance thermique installée
$P_{installée} = \frac{E_{finale}}{equivalent \: heures\:  pleine \: charge}$

In [10]:
ehpc = 2300
data["P_[kW]"] = round(data["E_utile_[kWh/an]"] / ehpc, 1)
data.head()

,TypeAffectation,PeriodeConstruction,AgentEnergetique,NbrNiveauxChauffes,EmpreinteSol_[m2],SRE_[m2],E_utile_[kWh/an],E_finale_[kWh/an],E_primaire_[kWh/an],RejCO2_[Tco2/an],P_[kW]
ID,,,,,,,,,,,
1,Immeuble de bureaux,1981-1985,Mazout,3,518,1398.6,176223.6,195804.0,242797.0,208.1,76.6
2,Batiment commercial,2011-2015,Mazout,1,1203,1082.7,42225.3,46917.0,58177.1,49.9,18.4
3,Immeuble de bureaux,1971-1980,Electricite,3,428,1155.6,179118.0,179118.0,419136.1,51.6,77.9
4,Immeuble de bureaux,1981-1985,Gaz naturel,4,985,3546.0,446796.0,496440.0,570906.0,431.1,194.3
5,Maison mitoyenne,1981-1985,Gaz naturel,4,1275,4590.0,711450.0,790500.0,909075.0,686.4,309.3


## 7. Bilan des installations existantes (et export vers csv)

In [11]:
data[["E_utile_[kWh/an]", "E_finale_[kWh/an]", "E_primaire_[kWh/an]", "RejCO2_[Tco2/an]", "P_[kW]"]].sum()

E_utile_[kWh/an]        8665325.1
E_finale_[kWh/an]       9559455.8
E_primaire_[kWh/an]    12280316.7
RejCO2_[Tco2/an]           9132.7
P_[kW]                     3767.5
dtype: float64

In [12]:
data.to_csv("resultats.csv")

## 8. Estimation des ressources disponibles
#### Blanchisserie

In [22]:
cp_eau = 4.185  # [kJ/kg.K]

dt_blanc = 45 - 12  # [deg.C]
flow_blanc = 5  #  [l/s]

p_blanc = dt_blanc * flow_blanc * cp_eau
print(int(p_blanc), 'kW')
print('Production totale:', int(p_blanc * 365 * 24)* 1e-6, 'GWh')

690 kW
Production totale: 6.048998999999999 GWh


#### UIOM

In [20]:
d = 5  # [h/j]
t_rej = 100  # [deg.C]
t_amb = 25  # [deg.C]
flow = 300  # [m3/h]
cp_fumees = 1.048  # [kJ/kg.K]

rho = 1.2  # [kg/m3]

p_uiom = flow * (t_rej - t_amb) * cp_fumees * rho / 3600
print(p_uiom, 'kW')
print('Production totale:', int(p_uiom * 5 * 365 * 9/12 * 1e-3), 'MWh')

7.86 kW
Production totale: 10 MWh


#### STEP

In [19]:
dt_hiver = 2  # [deg.C]
dt_ete = 6  # [deg.C]

flow_jour = 300  # [l/s]
flow_nuit = 50  # [l/s]

p_hiver_jour = dt_hiver * flow_jour * cp_eau
p_ete_jour = dt_ete * flow_jour * cp_eau
p_hiver_nuit = dt_hiver * flow_nuit * cp_eau
p_ete_nuit = dt_ete * flow_nuit * cp_eau

print('H+J:', int(p_hiver_jour), 'kW')
print('E+J:', int(p_ete_jour), 'kW')
print('H+N:', int(p_hiver_nuit), 'kW')
print('E+N:', int(p_ete_nuit), 'kW')

print('Production totale:', int(365/4 * 24 * (p_hiver_jour + p_ete_jour + p_hiver_nuit + p_ete_nuit) * 1e-6), 'GWh')

H+J: 2510 kW
E+J: 7532 kW
H+N: 418 kW
E+N: 1255 kW
Production totale: 25662 MWh


#### Solaire thermique

In [16]:
data.loc[data["EmpreinteSol_[m2]"] < 250]

,TypeAffectation,PeriodeConstruction,AgentEnergetique,NbrNiveauxChauffes,EmpreinteSol_[m2],SRE_[m2],E_utile_[kWh/an],E_finale_[kWh/an],E_primaire_[kWh/an],RejCO2_[Tco2/an],P_[kW]
ID,,,,,,,,,,,
15,Maison individuelle,1996-2000,Mazout,1,158,142.2,17064.0,18960.0,23510.4,20.1,7.4
17,Maison mitoyenne,1981-1985,Mazout,2,201,361.8,56079.0,62310.0,77264.4,66.2,24.4


In [17]:
irr = 3.54  # [kWh/m2/jour]

def e_solaire(row):
    if row["EmpreinteSol_[m2]"] > 250:
        return 0.7 * row["EmpreinteSol_[m2]"]
    else:
        return 0.4 * row["EmpreinteSol_[m2]"]
    
e_solaire = data.apply(e_solaire, axis=1)
print("Production totale", round(e_solaire.sum()*1e-3), "MWh")

Production totale 19.0 MWh
